In [5]:
import numpy as np
from scipy.optimize import minimize_scalar
from scipy.optimize import minimize

In [2]:
def steepest_descent(f, grad_f, x0, tol=1e-6, max_iter=100):
    """
    Steepest Descent Method with Exact Line Search

    Parameters:
    - f: function f(x), where x is a numpy array
    - grad_f: gradient function grad_f(x) returning numpy array
    - x0: initial point (numpy array)
    - tol: tolerance for stopping criterion (norm of gradient)
    - max_iter: maximum number of iterations

    Returns:
    - x: final point (minimum estimate)
    - history: list of iterates
    """
    x = x0.copy()
    history = [x.copy()]

    for i in range(max_iter):
        grad = grad_f(x)
        grad_norm = np.linalg.norm(grad)

        if grad_norm < tol:
            print(f"Converged in {i} iterations.")
            break

        d = -grad  # Steepest descent direction

        # function for line search: f_alpha(alpha) = arg_min(f(x + alpha*d)) for alpha>0
        f_alpha = lambda alpha: f(x + alpha * d)
        res = minimize_scalar(f_alpha)

        alpha = res.x  # Best value
        x = x + alpha * d
        history.append(x.copy())

        print(f"Iter {i+1}: alpha = {alpha:.6f}, x = {x}, f(x) = {f(x):.6f}, ||grad|| = {grad_norm:.6f}")

    return x, history


In [3]:

def f(x):
    x1, x2 = x
    return (x1 - 2)**2 + (x2 + 3)**2


def grad_f(x):
    x1, x2 = x
    return np.array([2*(x1 - 2), 2*(x2 + 3)])


x0 = np.array([0.0, 0.0])


x_min, history = steepest_descent(f, grad_f, x0)

print("\nMinimum found at:", x_min)
print("f(min) =", f(x_min))


Iter 1: alpha = 0.500000, x = [ 2. -3.], f(x) = 0.000000, ||grad|| = 7.211103
Converged in 1 iterations.

Minimum found at: [ 2. -3.]
f(min) = 0.0


In [39]:
def line_search(f, grad_f, x, d, c1=1e-4, rho=0.5):
    alpha = 1.0
    while f(x + alpha * d) > f(x) + c1 * alpha * np.dot(grad_f, d):
        alpha *= rho
    return alpha

#### BFGS (Built-in function)

In [132]:
def quadratic_function(x):
    Q = np.array([[2, -3], [-3, 5]])  
    b = np.array([0, -1])                              
    return 0.5 * np.dot(x, np.dot(Q, x)) - np.dot(b, x)

# Gradient of quadratic function
def gradient(x):
    Q = np.array([[2, -3], [-3, 5]])  
    b = np.array([0, -1])
    return np.dot(Q, x) - b

x0 = np.array([0.0, 0.0])

# Minimize 
result = minimize(quadratic_function, x0, method='BFGS', jac=gradient)

In [133]:
print("Optimal solution:", result.x)
print("Function value at optimum:", result.fun)
print("Number of iterations:", result.nit)

Optimal solution: [-3. -2.]
Function value at optimum: -0.9999999999999989
Number of iterations: 5


### BFGS 

In [40]:
def BFGS(f, grad_f, Q, x0, tol=1e-3):
    x = x0.copy()
    B = np.array([[1, 0], [0, 1]])
    H = np.linalg.inv(B)
    i=0
    while True:
        grad = grad_f(x)
        grad_norm = np.linalg.norm(grad)
        
        if grad_norm < tol:
            print(f"Converged in {i} iterations.")
            break
        d = -(H @ grad)
        
        #f_alpha = lambda alpha: f(x + alpha * d)
        #res = minimize_scalar(f_alpha)
        #alpha = res.x
        alpha = line_search(f, grad_f(x), x, d)
        del_x = alpha * d
        #B = B + (grad_f(x) @ grad_f(x).T)/(grad_f(x).T @ grad_f(x)) - ((B @ del_x) @ (del_x.T @ B))/(del_x.T @ B @ del_x)
        y = grad_f(x + alpha * d) - grad_f(x)
        B = B - ((B @ del_x) @ (del_x @ B))/(del_x @ (B @ del_x)) + (y @ y.T)/(y.T @ del_x)
        try:
            H = np.linalg.inv(B)
        except np.linalg.LinAlgError:
            print("Hessian matrix is singular.")
            B = np.eye(len(x0))
            H = np.linalg.inv(B)
        
        x = x - alpha * (H @ grad)
        print(f"Iter {i+1}: alpha = {alpha:.6f}, x = {x}, f(x) = {f(x):.6f}, ||grad|| = {grad_norm:.6f}")
        i+=1
    return x

In [42]:
def quadratic_function(x):
    Q = np.array([[4, -4], [-4, 8]])
    b = np.array([4, 0])
    c = 4
    return 0.5 * np.dot(x,np.dot(Q,x)) - np.dot(b,x) + c

# Gradient of the above quadratic function
def gradient(x):
    Q = np.array([[4, -4], [-4, 8]])  
    b = np.array([4, 0])
    return np.dot(Q, x) - b

x0 = np.array([0.0, 0.0])
Q = np.array([[4, -4], [-4, 8]])
x_min = BFGS(quadratic_function, gradient, Q, x0)

print("\nMinimum found at:", x_min)
print("f(min) =", quadratic_function(x_min))

Iter 1: alpha = 0.250000, x = [ 0.53333333 -0.46666667], f(x) = 4.302222, ||grad|| = 4.000000
Iter 2: alpha = 0.125000, x = [ 0.17808149 -0.08858518], f(x) = 3.445591, ||grad|| = 5.866667
Iter 3: alpha = 0.250000, x = [ 0.3814407  -0.26330746], f(x) = 3.444299, ||grad|| = 3.259403
Iter 4: alpha = 0.125000, x = [ 0.24982804 -0.11851823], f(x) = 3.300139, ||grad|| = 3.900295
Iter 5: alpha = 0.250000, x = [ 0.3610886 -0.1520469], f(x) = 3.128498, ||grad|| = 3.190043
Iter 6: alpha = 0.500000, x = [0.26923651 0.11273678], f(x) = 2.997457, ||grad|| = 3.297283
Iter 7: alpha = 0.125000, x = [ 0.47658673 -0.07978166], f(x) = 2.725475, ||grad|| = 3.378539
Iter 8: alpha = 0.250000, x = [0.4084323  0.04458236], f(x) = 2.635020, ||grad|| = 3.102247
Iter 9: alpha = 0.250000, x = [ 0.58522498 -0.09550743], f(x) = 2.604137, ||grad|| = 2.847086
Iter 10: alpha = 0.125000, x = [0.4786172  0.02637091], f(x) = 2.495975, ||grad|| = 3.357332
Iter 11: alpha = 0.500000, x = [0.70215992 0.00615695], f(x) = 2.16

### Q Conjugate (Directions Not given)

In [30]:
def Q_conjugate_without_directions(f, grad_f, x, Q, tol=1e-6):
    grad = grad_f(x)
    d = -grad
    i = 0
    while True:
        Qd = Q @ d
        alpha = -(grad.T @ d) / (d.T @ Qd)
        x = x + alpha * d
        grad_new = grad_f(x)
        grad_norm = np.linalg.norm(grad_new)
        print(f"Iter {i+1}: alpha = {alpha:.6f}, x = {x}, f(x) = {f(x):.6f}, ||grad|| = {grad_norm:.6f}, direction : {d}")
        if grad_norm < tol:
            print(f"Converged in {i+1} iterations.")
            break
        beta = (grad_new.T @ Qd) / (d.T @ Qd)
        d = -grad_new + beta * d
        grad = grad_new
        i += 1

In [32]:
def quadratic_function(x):
    Q = np.array([[4, -4], [-4, 8]])
    b = np.array([4, 0])
    c = 4
    return 0.5 * np.dot(x,np.dot(Q,x)) - np.dot(b,x) + c

# Gradient of the above quadratic function
def gradient(x):
    Q = np.array([[4, -4], [-4, 8]])  
    b = np.array([4, 0])
    return np.dot(Q, x) - b

x0 = np.array([0.0, 0.0])
Q = np.array([[4, -4], [-4, 8]])
Q_conjugate_without_directions(quadratic_function, gradient, x0, Q)

Iter 1: alpha = 0.250000, x = [1. 0.], f(x) = 2.000000, ||grad|| = 4.000000, direction : [ 4. -0.]
Iter 2: alpha = 0.250000, x = [2. 1.], f(x) = 0.000000, ||grad|| = 0.000000, direction : [4. 4.]
Converged in 2 iterations.


### Q Conjugate (Directions given)

In [33]:
def Q_conjugate_with_directions(f, grad_f, x, Q, d, tol=1e-6):
    for i in range(d.shape[0]):
        Qd = Q @ d[i]
        alpha = -(grad_f(x).T @ d[i])/(d[i].T @ Qd)
        grad_norm = np.linalg.norm(grad_f(x))
        if grad_norm < tol:
            print(f"Converged in {i+1} iterations.")
            break    
        x = x + alpha * d[i]
        print(f"Iter {i+1}: alpha = {alpha:.6f}, x = {x}, f(x) = {f(x):.6f}, ||grad|| = {grad_norm:.6f}")
    return x


In [34]:
def quadratic_function(x):
    Q = np.array([[4, -4], [-4, 8]])  
    b = np.array([4, 0])
    c = 4
    return 0.5 * np.dot(x,np.dot(Q,x)) - np.dot(b,x) + c

# Gradient of the above quadratic function
def gradient(x):
    Q = np.array([[4, -4], [-4, 8]])  
    b = np.array([4, 0])
    return np.dot(Q, x) - b

x0 = np.array([0.0, 0.0])
Q = np.array([[4, -4], [-4, 8]])
d = np.array([[1,0],[0,1]])
Q_conjugate_with_directions(quadratic_function, gradient, x0, Q, d)

Iter 1: alpha = 1.000000, x = [1. 0.], f(x) = 2.000000, ||grad|| = 4.000000
Iter 2: alpha = 0.500000, x = [1.  0.5], f(x) = 1.000000, ||grad|| = 4.000000


array([1. , 0.5])